In [15]:
import pickle
import json
from glob import glob

In [16]:
#pkl_list = glob("./gqa_data/*.pkl")

with open("./gqa_data/created_data.pkl" , "rb") as f:
    pkl = pickle.load(f)
    
with open("./gqa_data/train_balanced_questions.json") as f:
    questions = json.load(f)

In [23]:
from pprint import pprint

#pprint(pkl['2354786'])

pprint(pkl['2354786'][5])

#print(questions['2354786'])

[['Is',
  'the',
  'gray',
  'bus',
  'catcher',
  'to',
  'the',
  'right',
  'of',
  'the',
  'other',
  'car',
  'waiting',
  'or',
  'driving',
  '?'],
 [[['young', 'catcher'], [2, 4]], [['gray', 'bus'], [2, 4]]],
 [0, 0]]


In [1]:
import json
from glob import glob
from pprint import pprint
from tqdm import tqdm

In [2]:
with open("./gqa_data/train_balanced_questions.json") as f:
    questions = json.load(f)

In [3]:
with open("./sceneGraphs/train_sceneGraphs.json") as f:
    sceneGraphs = json.load(f)

In [4]:
def get_related_object_names(objects, related_objects, question):
    
    object_ids = {}
    
    question_annotated_objects = related_objects['question']
    '''
    for key, value in related_objects.items():
    
        object_ids[key] = set()
        for start_idx, object_id  in value.items():
            object_ids[key].add(object_id)
    '''
    object_ids['question'] = set()
    for start_idx, object_id in question_annotated_objects.items():
       
        object_ids['question'].add(object_id)
    
    object_names = []
    
    related_object_dict = {}
    related_object_ids = set()
    related_object_names = set()
    irrelated_object_dict = {}
    
    for key, object_ids in object_ids.items():
        related_object_dict[key] = {}
        for object_id in object_ids:
            object = objects[object_id]
            object_name = object['name']
            if object_name in question:
                related_object_dict[key][object_id] = object_name
                related_object_ids.add(object_id)
                related_object_names.add(object_name)
    
    for key, value in objects.items():
        
        object_name = value['name']
        if not key in related_object_ids and object_name in related_object_names:
            irrelated_object_dict[key] = object_name
    
    
    return related_object_dict, irrelated_object_dict

In [5]:
print(type(questions))

ambiguous_questions = {}
exsit_question_nums = 0

for idx, (question_key, question_value) in tqdm(enumerate(questions.items())):
    if idx < 1:
        pprint(question_value)
    #pprint(question_value)
    if 'exist' in question_value['types']['detailed']:
        exsit_question_nums += 1
        # if exsit_question_nums < 5:
        #    print("exist-question\n",question_value['question'])
        continue
    image_id = question_value['imageId']
    #pprint(sceneGraphs[image_id])
    sceneGraph=sceneGraphs[image_id]
    related_objects = question_value['annotations']
    objects = sceneGraph['objects']
    
    related_object_names, irrelated_object_names = get_related_object_names(objects, related_objects,question_value['question'] )
    #names_set = set(names)
    
    #if len(names) != len(names_set):
    if len(irrelated_object_names) > 0:
        if len(ambiguous_questions.keys()) < 30:
        #pprint(question_value)
        #pprint(sceneGraphs[image_id])
            print(image_id)
            pprint(related_object_names)
            pprint(irrelated_object_names)
            print(question_value['question'])
            print(question_value['fullAnswer'])
            print(question_value['types']['detailed'])
            print("---------------------------")
        #print('다름')
        #break
        question_value['irrelated_object_names'] = irrelated_object_names
        ambiguous_questions[question_key] = question_value

<class 'dict'>


10485it [00:00, 104842.10it/s]

{'annotations': {'answer': {},
                 'fullAnswer': {'2': '2486325'},
                 'question': {'2': '2486325'}},
 'answer': 'yes',
 'entailed': ['02930160',
              '02930158',
              '02930159',
              '02930154',
              '02930155',
              '02930156',
              '02930153'],
 'equivalent': ['02930152'],
 'fullAnswer': 'Yes, the sky is dark.',
 'groups': {'global': None, 'local': '06-sky_dark'},
 'imageId': '2354786',
 'isBalanced': True,
 'question': 'Is the sky dark?',
 'semantic': [{'argument': 'sky (2486325)',
               'dependencies': [],
               'operation': 'select'},
              {'argument': 'dark',
               'dependencies': [0],
               'operation': 'verify color'}],
 'semanticStr': 'select: sky (2486325)->verify color: dark [0]',
 'types': {'detailed': 'verifyAttr',
           'semantic': 'attr',
           'structural': 'verify'}}
2375429
{'question': {'722332': 'wall'}}
{'722333': 'wall'}
What is 

943000it [00:06, 140770.22it/s]


In [6]:
print(len(questions.keys()))
print(exsit_question_nums)
print(len(ambiguous_questions.keys()))

943000
163813
125854


In [8]:
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl",cache_dir="./blip2-flan-t5-xxl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", torch_dtype=torch.float16, cache_dir="./blip2-flan-t5-xxl")
model.to(device)

Loading checkpoint shards: 100%|██████████| 6/6 [02:59<00:00, 29.95s/it]


Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [13]:
new_dataset = {}
new_dataset_length = len(ambiguous_questions)
for idx, (question_id, question_value) in tqdm(enumerate(ambiguous_questions.items()), total=new_dataset_length):
    if idx == new_dataset_length:
        break
    
    image_id = question_value['imageId']
    img_file = './images/' + image_id + '.jpg' 
    raw_image = Image.open(img_file).convert('RGB')

    question = "Instructions: Given a picture, A question and a correct answer related the picture are provided.\
    The answer can be inferred from the picture. \
    The target(object) of question  which is existed in the picture is important key to infer the answer. (objects in question are bold) \
    The additional information of object related question is helpful for answer the question more correctly. \
    Therefore, our goal is to get new information related answer by asking a new question. \
    Write an additional question to help to answer the original question correctly.  \
    original question: " + question_value['question'] + " " + \
    "uncertain information: " + list(question_value['irrelated_object_names'].values())[0] + " " + \
    "answer: " + question_value['fullAnswer'] + " " + \
    "additional question: "

    if idx < 2:
        print(question)

    inputs = processor(raw_image, question, return_tensors="pt").to(device, torch.float16)
    out = model.generate(**inputs)
    additional_question = processor.decode(out[0], skip_special_tokens=True)
    
    question_value['addtional_question'] = additional_question
    new_dataset[question_id] = question_value
    
    

  0%|          | 0/125854 [00:00<?, ?it/s]/opt/conda/envs/torch2.0/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Instructions: Given a picture, A question and a correct answer related the picture are provided.    The answer can be inferred from the picture.     The target(object) of question  which is existed in the picture is important key to infer the answer. (objects in question are bold)     The additional information of object related question is helpful for answer the question more correctly.     Therefore, our goal is to get new information related answer by asking a new question.     Write an additional question to help to answer the original question correctly.      original question: What is on the white wall? uncertain information: wall answer: The pipe is on the wall. additional question: 


  0%|          | 1/125854 [00:00<14:57:18,  2.34it/s]

Instructions: Given a picture, A question and a correct answer related the picture are provided.    The answer can be inferred from the picture.     The target(object) of question  which is existed in the picture is important key to infer the answer. (objects in question are bold)     The additional information of object related question is helpful for answer the question more correctly.     Therefore, our goal is to get new information related answer by asking a new question.     Write an additional question to help to answer the original question correctly.      original question: Who is wearing a shirt? uncertain information: shirt answer: The girl is wearing a shirt. additional question: 


  2%|▏         | 2177/125854 [14:45<13:58:17,  2.46it/s]


KeyboardInterrupt: 

In [12]:
with open("ambiguous_questions.json", 'w') as f:
    json.dump(new_dataset, f, ensure_ascii=False)

In [4]:
import json
import random
import numpy as np

with open("ambiguous_questions.json", 'r') as f:
    full_dataset  = json.load(f)

dataset_sampling = {}

full_list = list(full_dataset.items())
np.random.seed(42)
random_indexs = np.random.choice(range(0, len(full_list)), 200, replace = False)
print(random_indexs)

[   127  81829  29650 104534  51049  13141  45881  97059  78613  25885
  50512  49529  36272 111730 115427 110915  51624  35578   3064  87015
 110087   2280  57527  18875  97344 100917  71409 109228 107489 112453
  42596  33438   6396 105189  44393  81737 118008 103008 103377  27884
  82236  96562  56427  43811  23451  74321 107619  53922  17415  68421
 105236  46251  25217   1472 104398  64769 114972 101514  59135  89220
  21776 117754  67387 118949  82389   4813  81420 118157  78634  73348
  31777  44235 106943    201  77919 108648   7863 104701  40922  59695
  22392 112274   9364 113949  55269  60947  84917   8277   1656 110292
  62789  30834  14186  91731 113576  73734  33495  58304  39929  51636
  70738    646  24418  71005  86838  84173   7520  96840  91868  68735
  88391  30096  59941  75132  96790 106741  13774  56504   2320  51937
  25058 108874  70206  19408   9994  20747 124396  54755  40008  73454
  29595  36842 124152  54919 120199  98710  54992 114971 125037  43006
  2868

In [6]:
import csv

train_list = [['question_id','question', 'answer', 'intermediate_question', 'ambiquous_object', 'image_id']]
test_list = [['question_id','question', 'answer', 'intermediate_question', 'ambiquous_object', 'image_id']]
for idx, q_idx in enumerate(random_indexs):
    (key, value) = full_list[q_idx] 
    dataset_sampling[key] = value
    original_q = value['question']
    answer = value['fullAnswer']
    ambiguous_object = list(value['irrelated_object_names'].values())[0]
    intermediate_q = value["addtional_question"]
    image_id = value['imageId']
    
    if idx < 100:
        train_list.append([q_idx, original_q, answer, intermediate_q, ambiguous_object, image_id])
    else:
        test_list.append([q_idx, original_q, answer, intermediate_q, ambiguous_object, image_id])
    
with open("ambiguous_questions_100_random.csv", 'w') as f:
#     json.dump(dataset_sampling, f, ensure_ascii=False)
    writer = csv.writer(f)
    writer.writerows(train_list)
    
with open("ambiguous_questions_test.csv", 'w') as f:
#     json.dump(dataset_sampling, f, ensure_ascii=False)
    writer = csv.writer(f)
    writer.writerows(test_list)

In [20]:
import csv
from datasets import load_dataset

dataset = load_dataset('csv',data_files={'train': "./ambiguous_questions_train.csv", 'test' : "./ambiguous_questions_test.csv"})

print(dataset['train'])

#with open("ambiguous_questions_100_random_label.csv", 'r') as f:
#    reader = csv.reader(f)
    
#    for line in reader:
#        print(line)
    

Using custom data configuration default-b4c98d92954de12c


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/envs/torch2.0/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b4c98d92954de12c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


/opt/conda/envs/torch2.0/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['question_id', 'question', 'answer', 'intermediate_question', 'ambiquous_object', 'image_id', 'is_ambiguous'],
    num_rows: 100
})


In [1]:
import pandas as pd
import json
import numpy as np

few_shot_examples = {}

intermediate_questions =  pd.read_excel("intermediate_questions_samples_32.xlsx", dtype={'qid' : str})

with open("./ambiguous_questions_rebuilt.json") as f:
    ambiguous_questions = json.load(f)

with open("./ambiguous_questions.json") as f:
    original_questions = json.load(f)

print(len(ambiguous_questions))
for idx in intermediate_questions.index:
    intermediate_question_example = intermediate_questions.iloc[idx]
    qid = intermediate_question_example['qid']

    ambiguous_question_example = ambiguous_questions[qid]
    
    ambiguous_question = ambiguous_question_example['question']
    intermediate_question = intermediate_question_example['intermediate question']
    original_question_example = original_questions.pop(qid)
    
    few_shot_examples[qid] = original_question_example
    few_shot_examples[qid]['ambiguous_question'] = ambiguous_question
    few_shot_examples[qid]['intermediate_question'] = intermediate_question
    few_shot_examples[qid].pop("addtional_question")
    
    entities_list = set()
    for object_name in few_shot_examples[qid]['irrelated_object_names'].values():
        if object_name in few_shot_examples[qid]['ambiguous_question']:
             entities_list.add(object_name)
    few_shot_examples[qid]['question_entities'] = list(entities_list)
   
np.random.seed(42) 
test_keys = np.random.choice(list(original_questions.keys()), 32, replace=False)
test_examples = {}

for qid in test_keys:
    original_questions[qid].pop("addtional_question")
    test_examples[qid] =  original_questions[qid]
    
    ambiguous_question_example = ambiguous_questions[qid]
    ambiguous_question = ambiguous_question_example['question']
    
    test_examples[qid]["ambiguous_question"] = ambiguous_question
    
    entities_list = set()
    for object_name in test_examples[qid]['irrelated_object_names'].values():
        if object_name in test_examples[qid]['ambiguous_question']:
             entities_list.add(object_name)
    test_examples[qid]['question_entities'] = list(entities_list)

125854


In [3]:
from pprint import pprint
test_examples_list = [{k:v}  for k,v in test_examples.items()]
pprint(test_examples_list[0])

{'12304166': {'ambiguous_question': "Is the man 's hair?",
              'annotations': {'answer': {},
                              'fullAnswer': {'1': '1035954'},
                              'question': {'2': '1035969', '3': '1035954'}},
              'answer': 'red',
              'entailed': [],
              'equivalent': ['12304166'],
              'fullAnswer': 'The hair is red.',
              'groups': {'global': 'color', 'local': '10c-hair_color'},
              'imageId': '4117',
              'irrelated_object_names': {'1035959': 'man',
                                         '1035961': 'hair',
                                         '1035973': 'man',
                                         '1035983': 'man'},
              'isBalanced': True,
              'question': "Is the man's hair red or light brown?",
              'question_entities': ['hair', 'man'],
              'semantic': [{'argument': 'man (1035969)',
                            'dependencies': [],
      